In [ ]:
# Save results to JSON (replicating original script functionality)
with open("method_out.json", "w") as f:
    json.dump(results, f, indent=2)
print("✅ Results saved to method_out.json")

print("\n" + "="*50)
print("🧪 EXPERIMENT WITH DIFFERENT PARAMETERS")
print("="*50)
print("Try changing these parameters in the DKWController and re-run:")
print("• epsilon_target: Target error threshold (default: 0.10)")
print("• delta: Confidence level (default: 0.05)")  
print("• min_samples: Minimum samples before deciding (default: 100)")
print("• hysteresis: Oscillation prevention margin (default: 0.05)")

# Example of parameter experimentation
print(f"\n📊 Quick parameter comparison:")
configs = [
    {"name": "Conservative", "epsilon_target": 0.05, "hysteresis": 0.02},
    {"name": "Aggressive", "epsilon_target": 0.15, "hysteresis": 0.08},
    {"name": "Quick Decision", "min_samples": 50},
]

for config in configs:
    # Create controller with modified parameters
    controller = DKWController(**config)
    test_results = {"proposed": []}
    
    for example in full_dataset[:50]:  # Use first 50 examples for quick test
        error = np.random.random() < example["difficulty"] 
        controller.add_observation(float(error))
        decision = controller.decide()
        test_results["proposed"].append({"decision": decision, "error": error})
    
    fusion_rate = sum(1 for r in test_results["proposed"] if r["decision"] == "fusion") / len(test_results["proposed"])
    error_rate = sum(r["error"] for r in test_results["proposed"]) / len(test_results["proposed"])
    
    print(f"  {config['name']}: {fusion_rate:.1%} fusion, {error_rate:.3f} error rate")

print(f"\n📝 The notebook is now self-contained and ready for experimentation!")

## Interactive Experimentation & Results Export

Try modifying the controller parameters and re-running the experiment to see how they affect performance. You can also save the results in the same format as the original script.

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Decision timeline
decisions_numeric = [1 if d == "fusion" else 0 for d in proposed_decisions]
axes[0,0].plot(decisions_numeric, 'b-', alpha=0.7, label='Proposed (DKW)')
axes[0,0].axhline(y=0, color='r', linestyle='--', alpha=0.7, label='Baseline (always fission)')
axes[0,0].set_ylabel('Decision (1=Fusion, 0=Fission)')
axes[0,0].set_xlabel('Example Index')
axes[0,0].set_title('Decision Timeline')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. Cumulative error rate
proposed_errors_cum = np.cumsum([r["error"] for r in proposed_results])
baseline_errors_cum = np.cumsum([r["error"] for r in baseline_results])
example_indices = range(1, len(proposed_results) + 1)

proposed_error_rate = proposed_errors_cum / np.array(example_indices)
baseline_error_rate = baseline_errors_cum / np.array(example_indices)

axes[0,1].plot(example_indices, proposed_error_rate, 'b-', label='Proposed (DKW)', alpha=0.8)
axes[0,1].plot(example_indices, baseline_error_rate, 'r-', label='Baseline', alpha=0.8)
axes[0,1].axhline(y=0.1, color='g', linestyle='--', alpha=0.7, label='Target (ε=0.1)')
axes[0,1].set_ylabel('Cumulative Error Rate')
axes[0,1].set_xlabel('Example Index')
axes[0,1].set_title('Error Rate Evolution')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# 3. Decision distribution
methods = ['Baseline', 'Proposed']
fission_counts = [baseline_decisions.count('fission'), proposed_decisions.count('fission')]
fusion_counts = [baseline_decisions.count('fusion'), proposed_decisions.count('fusion')]

x = np.arange(len(methods))
width = 0.35

axes[1,0].bar(x - width/2, fission_counts, width, label='Fission', alpha=0.8, color='red')
axes[1,0].bar(x + width/2, fusion_counts, width, label='Fusion', alpha=0.8, color='blue')
axes[1,0].set_ylabel('Number of Decisions')
axes[1,0].set_title('Decision Distribution')
axes[1,0].set_xticks(x)
axes[1,0].set_xticklabels(methods)
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# 4. DKW bound evolution (for last 100 samples)
controller_demo = DKWController()
dkw_bounds = []
empirical_errors = []
sample_sizes = []

for i, example in enumerate(full_dataset):
    error = np.random.random() < example["difficulty"]
    controller_demo.add_observation(float(error))
    
    if len(controller_demo.samples) >= controller_demo.min_samples:
        n = len(controller_demo.samples)
        epsilon = controller_demo.dkw_epsilon(n)
        empirical = np.mean(controller_demo.samples[-controller_demo.min_samples:])
        
        dkw_bounds.append(epsilon)
        empirical_errors.append(empirical)
        sample_sizes.append(n)

if dkw_bounds:  # Only plot if we have data
    axes[1,1].plot(sample_sizes, empirical_errors, 'b-', label='Empirical Error', alpha=0.8)
    axes[1,1].fill_between(sample_sizes, 
                          np.array(empirical_errors) - np.array(dkw_bounds),
                          np.array(empirical_errors) + np.array(dkw_bounds),
                          alpha=0.3, label='DKW Confidence Band')
    axes[1,1].axhline(y=0.1, color='g', linestyle='--', alpha=0.7, label='Target (ε=0.1)')
    axes[1,1].set_ylabel('Error Rate')
    axes[1,1].set_xlabel('Sample Size')
    axes[1,1].set_title('DKW Bound Evolution')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n=== PERFORMANCE SUMMARY ===")
print(f"DKW Controller efficiency: {fusion_counts[1]/len(proposed_results):.1%} fusion decisions")
print(f"Error rate difference: {(proposed_errors/len(proposed_results) - baseline_errors/len(baseline_results)):.3f}")
if dkw_bounds:
    print(f"Final DKW bound: ±{dkw_bounds[-1]:.3f}")
    print(f"Final empirical error: {empirical_errors[-1]:.3f}")

## Visualization and Analysis

Let's visualize the controller's behavior over time and analyze its performance.

In [ ]:
# Run the experiment
results = run_experiment(full_dataset)

# Display basic statistics
baseline_results = results["baseline"]
proposed_results = results["proposed"]

print("=== EXPERIMENT RESULTS ===")
print(f"Total examples processed: {len(baseline_results)}")
print()

# Count decisions for each approach
baseline_decisions = [r["decision"] for r in baseline_results]
proposed_decisions = [r["decision"] for r in proposed_results]

print("Baseline (always fission):")
print(f"  Fission decisions: {baseline_decisions.count('fission')}")
print(f"  Fusion decisions: {baseline_decisions.count('fusion')}")
print()

print("Proposed (DKW controller):")
print(f"  Fission decisions: {proposed_decisions.count('fission')}")
print(f"  Fusion decisions: {proposed_decisions.count('fusion')}")
print()

# Calculate error rates
baseline_errors = sum(r["error"] for r in baseline_results)
proposed_errors = sum(r["error"] for r in proposed_results)

print("Error rates:")
print(f"  Baseline error rate: {baseline_errors/len(baseline_results):.3f}")
print(f"  Proposed error rate: {proposed_errors/len(proposed_results):.3f}")

# Show first few results as examples
print(f"\nFirst 10 results (Proposed method):")
for i in range(10):
    r = proposed_results[i]
    print(f"  {r['id']}: {r['decision']} (error: {r['error']})")

## Run Experiment

Let's execute the experiment with our sample data and examine the results.

In [ ]:
def run_experiment(data):
    """Run DKW controller experiment with inlined data."""
    controller = DKWController()
    results = {"baseline": [], "proposed": []}

    for example in data:
        # Simulate error occurrence based on difficulty
        error = np.random.random() < example["difficulty"]
        controller.add_observation(float(error))
        decision = controller.decide()

        results["proposed"].append({
            "id": example["id"],
            "decision": decision,
            "error": error,
        })
        results["baseline"].append({
            "id": example["id"],
            "decision": "fission",  # Always conservative
            "error": error,
        })

    return results

## Experiment Function

The experiment function compares two approaches:
1. **Baseline**: Always uses conservative "fission" mode
2. **Proposed**: Uses the DKW controller to dynamically switch between fusion and fission based on observed error rates

The function simulates errors based on example difficulty and tracks decisions made by each approach.

In [ ]:
# Sample dataset - inlined to make notebook self-contained
# This replaces the need for external "../dataset_001/data_out.json"
sample_data = [
    {"id": "example_000", "difficulty": 0.05},  # Easy example
    {"id": "example_001", "difficulty": 0.08},  # Easy example  
    {"id": "example_002", "difficulty": 0.15},  # Medium difficulty
    {"id": "example_003", "difficulty": 0.03},  # Very easy
    {"id": "example_004", "difficulty": 0.12},  # Medium difficulty
    {"id": "example_005", "difficulty": 0.20},  # Hard example
    {"id": "example_006", "difficulty": 0.07},  # Easy example
    {"id": "example_007", "difficulty": 0.18},  # Medium-hard
    {"id": "example_008", "difficulty": 0.04},  # Easy example
    {"id": "example_009", "difficulty": 0.25},  # Very hard
]

# Extend the dataset for more realistic experimentation
extended_data = []
for i in range(200):
    # Generate examples with varying difficulty following a realistic distribution
    if i < 100:
        difficulty = np.random.beta(2, 10)  # Skewed towards easier examples
    else:
        difficulty = np.random.beta(3, 5)   # More mixed difficulty
    
    extended_data.append({
        "id": f"example_{i:03d}",
        "difficulty": min(max(difficulty, 0.01), 0.5)  # Clamp between 1% and 50%
    })

# Combine sample and extended data
full_dataset = sample_data + extended_data

print(f"Created dataset with {len(full_dataset)} examples")
print(f"Difficulty range: {min(ex['difficulty'] for ex in full_dataset):.3f} - {max(ex['difficulty'] for ex in full_dataset):.3f}")
print(f"Mean difficulty: {np.mean([ex['difficulty'] for ex in full_dataset]):.3f}")

## Sample Data

Instead of reading from external JSON files, we'll inline the sample data directly. The data consists of examples with varying difficulty levels that influence the probability of errors occurring.

In [ ]:
@dataclass
class DKWController:
    """DKW-guided fusion/fission controller."""
    epsilon_target: float = 0.10
    delta: float = 0.05
    min_samples: int = 100
    hysteresis: float = 0.05

    samples: list = field(default_factory=list)
    current_state: str = "fission"

    def dkw_epsilon(self, n: int) -> float:
        """Compute DKW epsilon for n samples."""
        if n < 2:
            return 1.0
        return np.sqrt(np.log(2 / self.delta) / (2 * n))

    def add_observation(self, error: float) -> None:
        """Add error observation for calibration."""
        self.samples.append(error)

    def decide(self) -> str:
        """Make fusion/fission decision with DKW guarantee."""
        n = len(self.samples)
        if n < self.min_samples:
            return self.current_state

        epsilon = self.dkw_epsilon(n)
        empirical_error = np.mean(self.samples[-self.min_samples:])
        error_upper_bound = empirical_error + epsilon

        if self.current_state == "fusion":
            if error_upper_bound > self.epsilon_target + self.hysteresis:
                self.current_state = "fission"
        else:
            if error_upper_bound < self.epsilon_target - self.hysteresis:
                self.current_state = "fusion"

        return self.current_state

## DKW Controller Class

The `DKWController` class implements a fusion/fission decision system with statistical guarantees. It uses the **Dvoretzky-Kiefer-Wolfowitz inequality** to bound the error between the empirical and true error distributions.

### Parameters:
- `epsilon_target`: Target error rate threshold (default: 0.10)
- `delta`: Confidence parameter for DKW bound (default: 0.05) 
- `min_samples`: Minimum samples before making decisions (default: 100)
- `hysteresis`: Prevents oscillation between modes (default: 0.05)

In [ ]:
"""Required imports for the DKW controller implementation."""
import json
import numpy as np
from dataclasses import dataclass, field
import matplotlib.pyplot as plt
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# DKW Controller Implementation Demo
## experiment_001: method.py

This notebook demonstrates a **DKW-guided fusion/fission controller** implementation. The controller uses the Dvoretzky-Kiefer-Wolfowitz (DKW) inequality to make statistically sound decisions about when to switch between fusion and fission modes based on error observations.

### Key Features:
- Statistical guarantees via DKW inequality
- Hysteresis to prevent mode switching oscillation  
- Configurable error tolerance and confidence levels